In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install PyPDF2
!pip install pdfminer
!pip install pdfplumber
!pip install pytesseract
!pip install pypdf
!pip install pdf2image
!apt-get install poppler-utils
!pip install opencv-python
! apt install tesseract-ocr
! apt install libtesseract-dev
!pip install ocrmypdf

In [ ]:
import PyPDF2
# To analyze the PDF layout and extract text
from pdfminer.high_level import extract_pages, extract_text
from pdfminer.layout import LTTextContainer, LTChar, LTRect, LTFigure
from pytesseract import Output, TesseractError

# To extract text from tables in PDF
import pdfplumber
import pytesseract
import nltk
from nltk.tokenize import sent_tokenize
nltk.download("punkt")
import pandas as pd
# To remove the additional created files
import os
import glob
import re
from pypdf import PdfReader
import numpy as np
import pdf2image
from pdf2image import convert_from_bytes
import cv2
pytesseract.pytesseract.tesseract_cmd = r'/bin/tesseract'


In [ ]:
from pdf2image import convert_from_path

pages = convert_from_path('20181012083108P030172521293001.pdf')

# Save each page as a JPEG file using Pillow

for i, page in enumerate(pages):
	page.save(f'page_{i}.jpg', 'JPEG')

In [ ]:
%cd /content/drive/My Drive/Colab Notebooks/



def extract_sch(doc_path, string_txt):
  reader = PdfReader(doc_path)

# get number of pages
  num_pages = len(reader.pages)

# define key terms

# extract text and do the search
  counter = 0
  index_val = 0
  df_pages = pd.DataFrame({'Doc_Id':[],'Page':[]})
  for page in reader.pages:
    counter = counter + 1
    text = page.extract_text()
    # print(text)
    res_search = re.search(string_txt, text)
    if res_search:
      index_val = index_val + 1
      current_df = pd.DataFrame({'Doc_Id': [doc_path],'Page': [counter]}, index = [index_val])
      df_pages = pd.concat([df_pages, current_df])
  return df_pages

#movado_g = extract_sch('20181004071217P040082195999001.pdf', "Schedule H, Line 4i")
northern_tst = extract_sch('20200729162134NAL0010984736001.pdf', "Schedule H, line 4(i)")
#finan_inst = extract_sch("20180724105202P030082018193001.pdf", "SCHEDULE H, LINE 4i")

#df_all = pd.concat([movado_g, northern_tst, finan_inst])


In [ ]:
print(northern_tst)

In [ ]:
from pdf2image import convert_from_path

%cd /content/drive/My Drive/Colab Notebooks/


# Replace 'input_file.pdf' with the path to your PDF file
pdf_file = '20181011071602P040246981943001.pdf'
pages = convert_from_path(pdf_file)

In [ ]:
def deskew(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.bitwise_not(gray)
    coords = np.column_stack(np.where(gray = 0))
    angle = cv2.minAreaRect(coords)[-1]

    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle

    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

    return rotated

In [ ]:
def extract_text_from_image(image):
    text = pytesseract.image_to_string(image)
    return text


In [ ]:
extracted_text = []
import cv2
pytesseract.pytesseract.tesseract_cmd = r'/bin/tesseract'  # your path may be different

for page in pages[24:29]:
    # Step 2: Preprocess the image (deskew)
    preprocessed_image = deskew(np.array(page))

    # Step 3: Extract text using OCR
    text = extract_text_from_image(preprocessed_image)
    extracted_text.append(text)


In [ ]:

def get_conf(page_gray):
    '''return a average confidence value of OCR result '''
    df = pytesseract.image_to_data(page_gray,output_type='data.frame')

    df.drop(df[df.conf==-1].index.values,inplace=True)
    df.reset_index()
    return df.conf.mean()

def deskew(image):
    '''deskew the image'''
    gray = cv2.bitwise_not(image)
    temp_arr = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    coords = np.column_stack(np.where(gray = 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated

'''
Main part of OCR:
pages_df: save eextracted text for each pdf file, index by page
OCR_dic : dict for saving df of each pdf, filename is the key
'''
pdf = '20200908153442NAL0011667955001.pdf'
#pages = convert_from_path(pdf)
file_list = [pdf]
OCR_dic={}
df = pd.DataFrame({'conf':[], 'text':[]})
for file in file_list:
    # convert pdf into image
    pdf_file = convert_from_bytes(open(os.path.join(file), 'rb').read())
    # create a df to save each pdf's text
    pages_df = pd.DataFrame(columns=['conf','text'])
    for (i,page) in enumerate(pdf_file[27:]) :
        try:
            # transfer image of pdf_file into array
            page_arr = np.asarray(page)
            print(page_arr)
            # transfer into grayscale
            page_arr_gray = cv2.cvtColor(page_arr,cv2.COLOR_BGR2GRAY)
            # deskew the page
            page_deskew = deskew(page_arr_gray)
            # cal confidence value
            page_conf = get_conf(page_deskew)
            # extract string

            pages_df = pd.DataFrame({'conf': page_conf,'text': pytesseract.image_to_string(page_deskew)}, index = ['A'])
            df = pd.concat([df, pages_df])
        except:
            # if can't extract then give some notes into df
            pages_df = pd.DataFrame({'conf': page_conf,'text': pytesseract.image_to_string(page_deskew)}, index = ['A'])
            df = pd.concat([df, pages_df])
            continue
    # save df into a dict with filename as key
    OCR_dic[file]=pages_df
    print('{} is done'.format(file))

In [ ]:
from pdf2image import convert_from_path
import glob
%cd /content/drive/My Drive/Colab Notebooks/


pdf = "20181011071602P040246981943001.pdf"
#pages = convert_from_path(pdf)
file_list = [pdf]
for pdf_path in file_list:
    pages = convert_from_path(pdf_path, 500)

    for pageNum,imgBlob in enumerate(pages):
        text = pytesseract.image_to_string(imgBlob,lang='eng')

        with open(f'{pdf_path[:-4]}_page{pageNum}.txt', 'w') as the_file:
            the_file.write(text)

In [ ]:
import pytesseract
from PIL import Image, ImageEnhance, ImageFilter
%cd /content/drive/My Drive/Colab Notebooks/

from pdf2image import convert_from_path

pages = convert_from_path('20200729162134NAL0010984736001.pdf')

# Save each page as a JPEG file using Pillow

for i, page in enumerate(pages):
	page.save(f'page_{i}.jpg', 'JPEG')


text = pytesseract.image_to_string(Image.open('page_48.jpg'))


In [ ]:

print(text)

In [ ]:
print(text)